In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import json
import numpy as np
import pandas as pd

In [ ]:
import sklearn as sk
import sklearn.linear_model as slm
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split ## Higher Version 0.19.1

In [ ]:
f = open('../input/result2/result2.json', encoding='utf8')
data = json.load(f)

In [ ]:
msgs = data['messages']
df = pd.DataFrame(msgs)
# df

In [ ]:
df_new = df.filter(items = ['id','from','text'])

#Eliminate all the messages by Tim, Raj, Dia and Trupti
df_new = df_new.loc[~df_new['from'].isin(['Tim','Raj','Trupti','Dia']) &  (df_new['text'].str.len() > 2)]
remove_list =['pin','wife','mrs','joined','whatsapp','good morning','good evening', 'gud nite', 
              'good night' 'good afternoon','dcotor','meant','hope','doctor','feature','qualification','forward']
df_new = df_new.loc[~df_new['text'].str.lower().str.contains('|'.join(remove_list),na=False)]
df_new

In [ ]:
#Retrieve all the satisfied messages

#List of words to be searched to retrieve replies to Tim and Raj 
include_list =['thank you','thanks','great','lucky','awesome','glad'] 
df_happy =  df_new.loc[df_new['text'].str.lower().str.contains('|'.join(include_list),na=False) , ['id','from',
                                                                                                   'text']]

exclude_list =['whatsapp','will try', 'darling','revert','which','what','shall','mr','medicine','try','video',
                'query','grinding','taste','colour','great day','practice','adding','bro','harmful','tense',
               'doctor','experience','install','imaginary','baby','exercise','fearing','joined','complaints',
               'clarifying','member','issue','accepting','order','line','afraid','weakness','hungry','manage',
               'tiredness','evidence','extreme','looking', 'mentality','prefer','opinion','officer','recipe',
               'roasted','thin','sprout','water','not well','reached','cooked','excercise', 'ask','avoid',
               'where','aspire','don''t','husband','eating','sud','hunger','answer',
              'hoping','planning','eaten','dia ','confusing']

#Eliminate all messages which has above words
df_happy = df_happy[~df_happy['text'].str.lower().str.contains('|'.join(exclude_list),na=False)]



In [ ]:
#Add a new column emotion and assign "1" for happy messages
df_happy["emotion"] = 1
df_happy

In [ ]:
#Retrieve all the Not Satisfied messages
#List of words to be searched to retrieve replies  to Tim and Raj

unhappy_list =['weakness','hungry','manage','tiredness','evidence','extreme','confusing']
df_unhappy = df_new.loc[df_new['text'].str.lower().str.contains('|'.join(unhappy_list),na=False) , ['id','from', 
                                                                                                    'text']]

#List of words to be searched in the df_unhappy dataframe to eliminate those questions
#Eliminating Happy texts and suggestions/advice from doctor members.
dropList = ['good morning','good evening','good afternoon','awesome','query','joined','thank','welcome',
            'great','congratulations','congrats','wow','motivational','please','good','perfect']

df_unhappy = df_unhappy.loc[~df_unhappy['text'].str.lower().str.contains('|'.join(dropList),na=False)]

#Add a new column emotion and assign "0" for unhappy messages
df_unhappy["emotion"] = 0
df_unhappy

In [ ]:
#Add a new column emotion and assign "0" for unhappy messages
df_unhappy["emotion"] = 0
df_unhappy

In [ ]:
#merge both happy and unhappy dataframes row-wise
dfs =[]
dfs.append(df_happy)
dfs.append(df_unhappy)

df_emotion = pd.concat(dfs)
df_emotion = df_emotion[['text','emotion']]
df_emotion

In [ ]:
#decision tree classifier to find whether an user is satisfied or not
X = df_emotion["text"]
Y = df_emotion["emotion"]

In [ ]:
#string columns cannot be used for prediction as algorithms only work on numeric data
#convert text column to numeric using countvectorizer
cv= CountVectorizer()

In [ ]:
# training the model with the available data(cleansed)

X_train,X_test,Y_train,Y_test = train_test_split(X,Y, test_size=0.50)

X_traincv = cv.fit_transform(X_train).toarray()

print(X_traincv.shape)


In [ ]:
#Decision tree classifier

model_ds = tree.DecisionTreeClassifier()
model_ds.fit(X_traincv,Y_train)

In [ ]:
X_testcv = cv.transform(X_test).toarray()

predict_ds = model_ds.predict(X_testcv)
# predict_ds
print('Predicted Value :',predict_ds[0])

In [ ]:
#find the accuracy

print("Accuracy:",metrics.accuracy_score(Y_test, predict_ds))

In [ ]:
#Linear regression  on Total messages Vs Satisfied
df_total = df_new.groupby('from')
df1 = df_total.count()
df1


In [ ]:
#get the count of satisfied msgs and have them in a dataframe
df_count = df_happy.groupby('from')
df2 = df_count.count()
df2

In [ ]:
result = pd.merge(df1, df2, left_on='from', right_on='from')
result[['text_x','emotion']]

In [ ]:
#assign x and y
x = result['text_x']
y= result['emotion']

LR = slm.LinearRegression()

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.25)
x_train = x_train.values.reshape(-1,1)
LR.fit(x_train, y_train)

In [ ]:
predict = LR.predict(x_test.values.reshape(-1,1))
predict

In [ ]:
print('Predicted Value :',predict[0])
print('Actual Value :',y_test.values[0])

In [ ]:
#Linear regression  on Total messages Vs Satisfied

plt.plot(x_test.values.reshape(-1,1), predict, color = 'green')
plt.show()

In [ ]:
#Linear regression  on Total messages Vs NotSatisfied
#get the count of notsatisfied msgs and have them in a dataframe
df_cnt = df_unhappy.groupby('from')
df3 = df_cnt.count()
df3

In [ ]:
result1 = pd.merge(df1, df3, left_on='from', right_on='from')
result1[['text_x','emotion']]

In [ ]:
#assign x and y
x1 = result['text_x']
y1= result['emotion']

LR1 = slm.LinearRegression()

In [ ]:
x1_train,x1_test,y1_train,y1_test = train_test_split(x1,y1,test_size=0.25)
x1_train = x1_train.values.reshape(-1,1)
LR1.fit(x1_train, y1_train)

In [ ]:
predict1 = LR1.predict(x1_test.values.reshape(-1,1))
predict1

In [ ]:
print('Predicted Value :',predict1[0])
print('Actual Value :',y1_test.values[0])

In [ ]:
#Linear regression  on Total messages Vs Satisfied

plt.plot(x1_test.values.reshape(-1,1), LR1.predict(x1_test.values.reshape(-1,1)), color = 'red')
plt.show()

In [ ]:
#retrieve top 10 satisfied based on the count

res = df2.nlargest(10,'id')
res[['text']]

In [ ]:
#retrieve top 10 unsatisfied based on the count

res1 = df3.nlargest(10,'id')
res1[['text']]

In [ ]:
#Add a new column emotion and assign "0" for all messages
df_new["emotion"] = 0
df_new

In [ ]:
df_new.update(df_happy) # this updated the column but now emotion col has float values
# df_new.to_csv("ques.csv") - For checking
#converting float column to int
df_new['emotion'] = df_new['emotion'].astype(int)
df_new

In [ ]:
#take only 2 columns text and emotion
df_new[['text', 'emotion']]